# 0/ Import

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import models
import torchvision.transforms.v2 as transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
import os
import time
import copy
import onnx
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, classification_report 
import numpy as np
from tqdm import tqdm 

import matplotlib.pyplot as plt

In [2]:
torch.__version__

'2.5.1+cu121'

# I/ Configuration and Hyperparameters

In [ ]:
DATA_DIR = r'**yourpath**' # Path to your dataset root folder
CSV_FILE = os.path.join(DATA_DIR, 'labels_daisee_continous.csv') # Path to your label CSV file
IMAGE_DIR = os.path.join(DATA_DIR, 'Dataset') # Path to the folder containing the images

# Define your emotions here, in the same order as your columns in the CSV
EMOTION_LABELS = ['boredom', 
                  'confusion',
                  'engagement', 
                  'frustration'
]

NUM_CLASSES = len(EMOTION_LABELS)

BATCH_SIZE = 32 
NUM_EPOCHS = 20
LEARNING_RATE = 0.001
FREEZE_FEATURES = True # True to fine-tune only the last layer, False to fine-tune the entire model

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Utilisation du périphérique : {DEVICE}")

Utilisation du périphérique : cuda:0


# II/ Creating the Custom PyTorch Dataset

In [ ]:
class EmotionDataset(Dataset):
    def __init__(self, df, img_dir, emotion_cols, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.emotion_cols = emotion_cols

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['image_path'])

        emotion_labels = row[self.emotion_cols].values.astype(float)
        emotion_labels = torch.tensor(emotion_labels, dtype=torch.float32)

        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, emotion_labels

# III/ Image transformations

In [ ]:
# Transformations for training (data augmentation)
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224), # Random crop and resize to 224x224
        transforms.RandomHorizontalFlip(), 
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1), 
        transforms.ToTensor(), 
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    # Transformations for validation/testing (just resizing and normalization)
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

c:\Users\alber\anaconda3\envs\dl_project_py311\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


# IV/ Data loading and Train/Val division

In [ ]:
print("Chargement des données...")
full_df = pd.read_csv(CSV_FILE)

# Splitting the dataset into training and validation sets
train_df, val_df = train_test_split(full_df, test_size=0.2, random_state=42)

train_dataset = EmotionDataset(df=train_df, img_dir=IMAGE_DIR, emotion_cols=EMOTION_LABELS, transform=data_transforms['train'])
val_dataset = EmotionDataset(df=val_df, img_dir=IMAGE_DIR, emotion_cols=EMOTION_LABELS, transform=data_transforms['val'])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

dataloaders = {'train': train_loader, 'val': val_loader}
dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}

print(f"Taille du dataset d'entraînement : {dataset_sizes['train']}")
print(f"Taille du dataset de validation : {dataset_sizes['val']}")

Chargement des données...
Taille du dataset d'entraînement : 6335
Taille du dataset de validation : 1584


# V/ Loading the pre-trained model

In [ ]:
print("Chargement et modification du modèle pré-entraîné...")

# 1. Loading the pre-trained model on ImageNet

# choice 1 => Resnet18
# model_ft = models.resnet18(weights="IMAGENET1K_V1")

# choice 2 => efficentnet
model_ft = models.efficientnet_b4(weights="IMAGENET1K_V1")

# # choice 3 => mobile net
# model_ft = models.mobilenet_v2(weights='IMAGENET1K_V1')

# choice 4 => efficentnet
# model_ft = models.efficientnet_b7(weights="IMAGENET1K_V1")

# choice 5  bis => convex_tiny 


print("Freeze des couches convolutionnelles...")

# 2. We freeze the feature extraction layers
for param in model_ft.parameters():
    param.requires_grad = False

# fine_tune_at = len(list(model_ft.children())) - 10
for name, param in list(model_ft.named_parameters())[-17:]:
    print(name)
    param.requires_grad = True

print("Définition de la dernière couche...")

# 3. Replacing the final fully-connected layer

# Try 1 => Resnet18
# num_ftrs = model_ft.fc.in_features

# model_ft.fc = nn.Sequential(
#     nn.Linear(num_ftrs, 512),
#     nn.BatchNorm1d(512),
#     nn.ReLU(),
#     nn.Dropout(0.1),
#     nn.Linear(512, NUM_CLASSES)
# )

# Try 2 => Efficientnet 
num_ftrs = model_ft.classifier[1].in_features

model_ft.classifier = nn.Sequential(
    nn.Linear(num_ftrs, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(512, NUM_CLASSES)
)

# 4. Sending the model to the correct device (GPU or CPU)
model_ft = model_ft.to(DEVICE)

print("Modèle prêt ! ✅")


Chargement et modification du modèle pré-entraîné...
Freeze des couches convolutionnelles...
features.7.1.block.0.1.weight
features.7.1.block.0.1.bias
features.7.1.block.1.0.weight
features.7.1.block.1.1.weight
features.7.1.block.1.1.bias
features.7.1.block.2.fc1.weight
features.7.1.block.2.fc1.bias
features.7.1.block.2.fc2.weight
features.7.1.block.2.fc2.bias
features.7.1.block.3.0.weight
features.7.1.block.3.1.weight
features.7.1.block.3.1.bias
features.8.0.weight
features.8.1.weight
features.8.1.bias
classifier.1.weight
classifier.1.bias
Définition de la dernière couche...
Modèle prêt ! ✅


In [14]:
from torchinfo import summary

summary(model_ft, input_size=(1, 3, 224, 224))

Layer (type:depth-idx)                                  Output Shape              Param #
EfficientNet                                            [1, 4]                    --
├─Sequential: 1-1                                       [1, 1792, 7, 7]           --
│    └─Conv2dNormActivation: 2-1                        [1, 48, 112, 112]         --
│    │    └─Conv2d: 3-1                                 [1, 48, 112, 112]         (1,296)
│    │    └─BatchNorm2d: 3-2                            [1, 48, 112, 112]         (96)
│    │    └─SiLU: 3-3                                   [1, 48, 112, 112]         --
│    └─Sequential: 2-2                                  [1, 24, 112, 112]         --
│    │    └─MBConv: 3-4                                 [1, 24, 112, 112]         (2,940)
│    │    └─MBConv: 3-5                                 [1, 24, 112, 112]         (1,206)
│    └─Sequential: 2-3                                  [1, 32, 56, 56]           --
│    │    └─MBConv: 3-6                    

# VI/ Loss Function and Optimizer


fonction expérimentale de pondération des classes pour contrebalancer le manque dans la distribution

In [ ]:
# def weighted_mse_loss(predictions, labels, weight_vector):
#     # get squared error on predictions
#     squared_diff = (predictions - labels)**2

#     # find which labels are positive or not (this will define which losses we weight)
#     positive_label_mask = (labels > 0)
    
#     # set weight to 1 if mask == 0 and weight otherwise
#     weights = torch.where(positive_label_mask == 1, weight_vector, 1)

#     weighted_squared_diff = weights * squared_diff

#     return torch.mean(weighted_squared_diff)

--- Fonction de perte et Optimiseur ---

In [ ]:
criterion_emotions = nn.MSELoss()
# criterion_emotions = weighted_mse_loss => to try

# Only parameters that require gradients will be optimized
optimizer_ft = optim.Adam(model_ft.parameters(), lr=LEARNING_RATE)

# Learning rate scheduler (reduces LR after a certain number of epochs)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# VII/ Training and assessment function

In [ ]:

def train_model(model, 
                criterion_emotions, 
                optimizer, 
                train_loader, 
                val_loader, 
                device, 
                scheduler=None,
                epochs=NUM_EPOCHS,
                output_names=EMOTION_LABELS):
    
    since = time.time()
    
    history = {'train_loss': [],
                'val_loss': [],
                'val_MAE': [],
                'val_MSE': [],
                'val_RMSE': [],
                'val_R2': []}

    model.to(device)

    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        print('-' * 30)

        ### -------- TRAIN --------
        model.train()
        running_loss = 0.0


        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion_emotions(outputs, labels)
            loss.backward()
            optimizer.step()

            # Accumulate loss
            running_loss += loss.item() * inputs.size(0)
            

        epoch_loss = running_loss / len(train_loader.dataset)
        history['train_loss'].append(epoch_loss)

        ### -------- VAL --------
        model.eval()
        val_loss = 0.0
        val_predictions = []
        val_true_values = []
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion_emotions(outputs, labels)
                val_loss += loss.item() * inputs.size(0)

                # Convert tensors to numpy to calculate sklearn metrics
                val_predictions.extend(outputs.cpu().numpy())
                val_true_values.extend(labels.cpu().numpy())

                preds = torch.round(outputs)
                preds = torch.clip(preds, 0, 3)

                all_preds.append(preds.cpu())
                all_labels.append(labels.cpu())

        # Concatenation of all batches
        all_preds = torch.cat(all_preds).numpy()
        all_labels = torch.cat(all_labels).numpy()

        # Convert lists to numpy array for sklearn
        val_true_values_np = np.array(val_true_values)
        val_predictions_np = np.array(val_predictions)

        val_epoch_loss = val_loss / len(val_loader.dataset)
        val_epoch_MAE = mean_absolute_error(val_true_values_np, val_predictions_np)
        val_epoch_MSE = mean_squared_error(val_true_values_np, val_predictions_np)
        val_epoch_RMSE = np.sqrt(mean_squared_error(val_true_values_np, val_predictions_np))
        val_epoch_r2 = r2_score(val_true_values_np, val_predictions_np)
        
        history['val_loss'].append(val_epoch_loss)
        history['val_MAE'].append(val_epoch_MAE)
        history['val_MSE'].append(val_epoch_MSE)
        history['val_RMSE'].append(val_epoch_RMSE)
        history['val_R2'].append(val_epoch_r2)
       
        print(f"Train Loss: {epoch_loss:.4f} | "
            f"Val Loss: {val_epoch_loss:.4f} | "
            f"Val MAE: {val_epoch_MAE:.4f} | "
            f"Val MSE: {val_epoch_MSE:.4f} | "
            f"Val RMSE: {val_epoch_RMSE:.4f} | "
            f"Val R2: {val_epoch_r2:.4f} \n ")
        
        # Displaying the multi-label classification report
        print("\n📊 Rapport de classification multilabels-multiouputs :")
        
        target_names_classes = ['0', '1', '2', '3'] # Class names for the report

        for i, names in enumerate(output_names):
            print(f"\n--- Variable {names} ---")
            
            # The real classes for the variable i
            y_true_var_i = all_labels[:, i]
            
            # The classified predictions for variable i
            y_pred_var_i = all_preds[:, i]
            
            # Check that the values are integers and within the expected range
            if not np.all(np.isin(y_true_var_i, [0, 1, 2, 3])):
                print(f"Attention: y_true pour la variable {names} contient des valeurs hors de [0, 3] ou non entières après arrondi.")
            if not np.all(np.isin(y_pred_var_i, [0, 1, 2, 3])):
                print(f"Attention: y_pred pour la variable {names} contient des valeurs hors de [0, 3] ou non entières après arrondi.")

            # Generate and display the report.
            print(classification_report(y_true_var_i, y_pred_var_i, target_names=target_names_classes, zero_division=0))
        

        if scheduler:
            scheduler.step()

    time_elapsed = time.time() - since
    print(f"\n🕒 Entraînement terminé en {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s")

    return history


# VIII/ Training Start

=> 15 epochs for the same results as us 

In [ ]:
# print("\nDébut de l'entraînement...\n")
# history = train_model(model=model_ft,
#                         criterion_emotions=criterion_emotions,
#                         optimizer=optimizer_ft,
#                         train_loader=train_loader,
#                         val_loader=val_loader,
#                         device=DEVICE,
#                         scheduler=exp_lr_scheduler,
#                         epochs=2,
#                         output_names=EMOTION_LABELS)



Début de l'entraînement...


Epoch 1/2
------------------------------
Train Loss: 0.6709 | Val Loss: 0.4850 | Val MAE: 0.5484 | Val MSE: 0.4850 | Val RMSE: 0.6964 | Val R2: 0.0267 
 

📊 Rapport de classification multilabels-multiouputs :

--- Variable boredom ---
              precision    recall  f1-score   support

           0       0.57      0.28      0.37       658
           1       0.32      0.75      0.45       486
           2       0.55      0.17      0.26       371
           3       0.00      0.00      0.00        69

    accuracy                           0.39      1584
   macro avg       0.36      0.30      0.27      1584
weighted avg       0.46      0.39      0.35      1584


--- Variable confusion ---
              precision    recall  f1-score   support

           0       0.76      0.77      0.77      1087
           1       0.33      0.43      0.37       361
           2       0.00      0.00      0.00       114
           3       0.00      0.00      0.00        22



Visualisation

In [18]:
from plotly import graph_objects as go

fig = go.Figure(data=[
                      go.Scatter(
                          y=history["train_loss"],
                          name="Training loss",
                          mode="lines"
                          ),
                      go.Scatter(
                          y=history["val_loss"],
                          name="Validation loss",
                          mode="lines"
                          )
])
fig.update_layout(
    title='Training and val loss across epochs',
    xaxis_title='epochs',
    yaxis_title='MSELoss()'    
)
fig.show()

In [19]:
from plotly import graph_objects as go

fig = go.Figure(data=[
                      go.Scatter(
                          y=history["val_MAE"],
                          name="val_MAE",
                          mode="lines"
                          ),
                      go.Scatter(
                          y=history["val_MSE"],
                          name="val_MSE",
                          mode="lines"
                          ),
                          go.Scatter(
                          y=history["val_RMSE"],
                          name="val_RMSE",
                          mode="lines"
                          ),
                      go.Scatter(
                          y=history["val_R2"],
                          name="val_R2",
                          mode="lines"
                          )
])
fig.update_layout(
    title='Score across epochs',
    xaxis_title='epochs',
    yaxis_title='Score'    
)
fig.show()

# IX/ Model registration + export

In [ ]:
# # Backup path
# onnx_export_path = "daisee_model.onnx"

# # Dummy input — must match the size expected by your model
# dummy_input = torch.randn(1, 3, 224, 224, device=DEVICE)

# # Export ONNX
# torch.onnx.export(
#     model_ft,                  
#     dummy_input,               
#     onnx_export_path,          
#     input_names=['input'],     
#     output_names=['output'],   
#     dynamic_axes={
#         'input': {0: 'batch_size'},
#         'output': {0: 'batch_size'}
#     },
#     opset_version=11,          # ONNX opset version (11 is safe for compatibility)
#     do_constant_folding=True   # Optimization for constants
# )

# print(f"✅ Modèle exporté au format ONNX : {onnx_export_path}")


✅ Modèle exporté au format ONNX : daisee_model.onnx
